In [1]:
import pandas as pd
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [10]:
import dvc.api

source =  dvc.api.get_url("data/Example_data.csv",
    repo="https://github.com/Amanuel3065/Prompt_for_jobDescription")


In [12]:

df = pd.read_csv('../data/Example_data.csv')


In [14]:
from csv import list_dialects


sentences_train,sentences_test,labels_train,labels_test = train_test_split(list(df['Body']),
                                                                           list(df['Analyst_Average_Score']),
                                                                           test_size=0.25, random_state=0)

In [15]:
#Get Embeddings

import creds

co = cohere.Client(creds.api_key)

In [16]:
#embed the training set
embeddings_train = co.embed(texts=sentences_train,model="large",truncate="LEFT").embeddings

#embed the testing set
embeddings_test = co.embed(texts=sentences_test,model="large",truncate="LEFT").embeddings

In [17]:
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Review text: Global and Regional Beta-Carotene Market Research 2020 Report | Growth Forecast 2025 key players! – DSM – BASF – Allied Biotech – Chr Hansen – LYCORED Beta-Carotene Market Global Beta-Carotene market provides an in-depth insight into the global market covering all its important chapters which are must read for key industry stakeholders. Report coverage include historical market performance, impact of COVID-19, post-covid scenario of market, forecast for six years, market dynamics including growth drivers, challenges, market opportunities, Porter’s five forces analysis, value chain analysis. It offers a detailed analysis for establishing value chain and market product across other countries. It also presents detailed insights of market segments, their future growth potential and analysis of countries. In addition to the above, the information is based on the partners that are highly competitive, key players as well as their market revenue in the forecast years of 2020 to 20

In [19]:
##Train

## import 
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced'))


In [26]:
#fit the svm, but first encode the continious categorical variable

from sklearn import preprocessing
from sklearn import utils

#convert train labels values to categorical values
lab = preprocessing.LabelEncoder()
labels_train_cat = lab.fit_transform(labels_train)

#convert test label values to categorical values
lab = preprocessing.LabelEncoder()
labels_test_cat = lab.fit_transform(labels_test)

#view transformed values
print(labels_train_cat)
print(labels_test_cat)

[0 0 0 2 0 0 1]
[0 1 0]


In [23]:

svm_classifier.fit(embeddings_train, labels_train_cat)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced'))])

In [27]:
#get score from test set
score = svm_classifier.score(embeddings_test,labels_test_cat)
print(f"Validation accuracy on large is {100*score}%!")

Validation accuracy on large is 66.66666666666666%!
